# LangChain: Memory
To augment our agents with the previous history and have a conversational flow.
## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory
Save the whole conversation.

In [1]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [4]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True # to see what is the happening internally in langchain, turn it to false to see the output only.
)

/var/folders/5v/819w4m8n69l11gbckr3pf5s00000gn/T/ipykernel_84948/472558519.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.0, model=llm_model)
/var/folders/5v/819w4m8n69l11gbckr3pf5s00000gn/T/ipykernel_84948/472558519.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/var/folders/5v/819w4m8n69l11gbckr3pf5s00000gn/T/ipykernel_84948/472558519.py:4: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2

In [5]:
conversation.predict(input="Hi, my name is Mohamed")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Mohamed
AI:

> Finished chain.


"Hello Mohamed! It's nice to meet you. How can I assist you today?"

In [6]:
conversation.predict(input="What is the color of the sun?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Mohamed
AI: Hello Mohamed! It's nice to meet you. How can I assist you today?
Human: What is the color of the sun?
AI:

> Finished chain.


"The color of the sun appears to be yellow when viewed from Earth's surface due to the scattering of sunlight in the Earth's atmosphere. However, the sun is actually white in color when viewed from space."

In [7]:
conversation.predict(input="thanls, now What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Mohamed
AI: Hello Mohamed! It's nice to meet you. How can I assist you today?
Human: What is the color of the sun?
AI: The color of the sun appears to be yellow when viewed from Earth's surface due to the scattering of sunlight in the Earth's atmosphere. However, the sun is actually white in color when viewed from space.
Human: thanls, now What is my name?
AI:

> Finished chain.


'Your name is Mohamed.'

In [8]:
# to see the memory content so far.
print(memory.buffer)

Human: Hi, my name is Mohamed
AI: Hello Mohamed! It's nice to meet you. How can I assist you today?
Human: What is the color of the sun?
AI: The color of the sun appears to be yellow when viewed from Earth's surface due to the scattering of sunlight in the Earth's atmosphere. However, the sun is actually white in color when viewed from space.
Human: thanls, now What is my name?
AI: Your name is Mohamed.


In [9]:
memory.load_memory_variables({})

{'history': "Human: Hi, my name is Mohamed\nAI: Hello Mohamed! It's nice to meet you. How can I assist you today?\nHuman: What is the color of the sun?\nAI: The color of the sun appears to be yellow when viewed from Earth's surface due to the scattering of sunlight in the Earth's atmosphere. However, the sun is actually white in color when viewed from space.\nHuman: thanls, now What is my name?\nAI: Your name is Mohamed."}

In [10]:
memory = ConversationBufferMemory()

In [11]:
# to append an explicit history to my buffer memory.
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [12]:
print(memory.buffer)

Human: Hi
AI: What's up


In [13]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

## ConversationBufferWindowMemory
Save a window from the conversation to save tokens and the respect the llm context window when the conversation get longer.

In [14]:
from langchain.memory import ConversationBufferWindowMemory

In [15]:
# k  represent the number of the saved exchange, which means one turn from the user and the other turn from the ChatBot
memory = ConversationBufferWindowMemory(k=1)               

/var/folders/5v/819w4m8n69l11gbckr3pf5s00000gn/T/ipykernel_84948/3627990533.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


In [16]:
memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [17]:
# memory only remembers the last pair only because we constrain k=1.
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [18]:
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=2)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [19]:
conversation.predict(input="Hi, my name is Mohamed")

"Hello Mohamed! It's nice to meet you. How can I assist you today?"

In [20]:
conversation.predict(input="What is 1+1?")

'1 + 1 equals 2. Is there anything else you would like to know?'

In [21]:
conversation.predict(input="What is my name?")

'Your name is Mohamed. How can I help you further, Mohamed?'

## ConversationTokenBufferMemory

Limit the memory to certain number of tokens instead of turns since most of LLM pricing is based on tokens.


In [22]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [ ]:
# we include the llm here to make sure our memory use the same tokenization technique as my model
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

/var/folders/5v/819w4m8n69l11gbckr3pf5s00000gn/T/ipykernel_84948/941208617.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)


In [24]:
memory.load_memory_variables({})

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory
Instead of limiting our memory to number of turns or tokens. We use another LLM to summarize our conversation so far and include the summary in the new turn as our conversational behavior.

In [25]:
from langchain.memory import ConversationSummaryBufferMemory

In [26]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

/var/folders/5v/819w4m8n69l11gbckr3pf5s00000gn/T/ipykernel_84948/1387621936.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)


In [27]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI.'}

In [28]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [29]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI.
Human: What would be a good demo to show?
AI:

> Finished chain.


'For the meeting with the product team, a good demo to show would be the latest prototype of the LangChain project, highlighting its key features and functionalities. This could include a live demonstration of the language translation capabilities, the user interface design, and any unique selling points that set LangChain apart from other language translation tools. Additionally, showcasing any recent updates or improvements made to the project would be beneficial to keep the team informed and engaged.'

In [30]:
memory.load_memory_variables({})

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The human asks what would be a good demo to show.\nAI: For the meeting with the product team, a good demo to show would be the latest prototype of the LangChain project, highlighting its key features and functionalities. This could include a live demonstration of the language translation capabilities, the user interface design, and any unique selling points that set LangChain apart from other language translation tools. Additionally, showcasing any recent updates or improvements made to the project would be beneficial to keep the team informed and engaged.'}